In [1]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
import sys
sys.path.append('/home/benedikt/Dokumente/parabolische_inverse_probleme')
from pymor.core.logger import set_log_levels
    
from pymor.basic import *
from problems import whole_problem
from discretizer import discretize_instationary_IP
from pymor.parameters.base import ParameterSpace

from model import InstationaryModelIP


In [2]:
set_log_levels({
    'pymor' : 'WARN'
})

In [3]:
N = 10                                                                         # FE Dofs = (N+1)^2                                                
noise_level = 1e-5        
nt = 10
fine_N = 2 * N

dims = {
    'N': N,
    'nt': nt,
    'fine_N': fine_N,
    'state_dim': (N+1)**2,
    'fine_state_dim': (fine_N+1)**2,
    'diameter': np.sqrt(2)/N,
    'fine_diameter': np.sqrt(2)/fine_N,
    'par_dim': (N+1)**2,
    'output_dim': 1,                                                                                                                                                                         # options to preassemble affine components or not
}

bounds = [0.001*np.ones((dims['par_dim'],)), 10e2*np.ones((dims['par_dim'],))]

model_parameter = {
    'T_initial' : 0,
    'T_final' : 1,
    'noise_percentage' : None,
    'noise_level' : 0.05,
    'q_circ' : 3*np.ones((nt, dims['par_dim'])), 
    'q_exact' : None,
    'bounds' : bounds,
    #'parameter_space' : ParameterSpace(analytical_problem.parameters, bounds) 
    'parameters' : None
}

optimizer_parameter = {
    
}

In [4]:
print('Construct problem..')                                                     
analytical_problem, q_exact, N, problem_type, exact_analytical_problem, energy_problem = whole_problem(
                                                        N = N,
                                                        parameter_location = 'reaction',
                                                        boundary_conditions = 'dirichlet',
                                                        exact_parameter = 'Kirchner',
                                                       )


Construct problem..


In [5]:
model_parameter['q_exact'] = q_exact
model_parameter['parameters'] = analytical_problem.parameters


In [6]:
print('Discretizing problem...')                                                
# discretize analytical problem to obtain inverse problem fom
building_blocks = discretize_instationary_IP(analytical_problem,
                            model_parameter,
                            dims, 
                            problem_type
                        ) 

model = InstationaryModelIP(
    *building_blocks,
    dims = dims,
    model_parameter = model_parameter
)


Discretizing problem...
noise percentage is 0.09589269187583464
noise_level is 0.05


In [7]:
#q_0 = []
q_exact = []
for idx in range(dims['nt']):
    #q_0.append(model_parameter['q_exact'] + 10 * idx)
    q_exact.append(model_parameter['q_exact'])
q_exact = np.array(q_exact)
q_0 = q_exact.copy() 
# q_0[:,0] += 1


In [8]:
# def compute(q):
#     u = model.solve_state(q)
#     p = model.solve_adjoint(q, u)
#     J = model.objective(u)
#     nabla_J = model.gradient(u,p)
#     return J, nabla_J

# q = q_0
# for i in range(10000):
#     J, nabla_J = compute(q)
#     d -= nabla_J.to_numpy()
#     print(J)

In [13]:
q = q_0
d = q.copy()

def compute(q, d, alpha):
    u = model.solve_state(q)
    p = model.solve_adjoint(q, u)
    
    lin_u = model.solve_linearized_state(q, d, u)
    lin_p = model.solve_linearized_adjoint(q, u, lin_u)
    
    lin_J = model.linearized_objective(q, d, u, lin_u, alpha)
    lin_nabla_J = model.linearized_gradient(q, d, u, lin_u, alpha)
    return lin_J, lin_nabla_J

lin_J, lin_nabla_J = compute(q, d, 1e-2)
print(lin_J)

0.3982446563756364


In [14]:
def compute(q, d, alpha):
    u = model.solve_state(q)
    p = model.solve_adjoint(q, u)
    
    lin_u = model.solve_linearized_state(q, d, u)
    lin_p = model.solve_linearized_adjoint(q, u, lin_u)
    
    lin_J = model.linearized_objective(q, d, u, lin_u, alpha)
    lin_nabla_J = model.linearized_gradient(q, d, u, lin_u, alpha)
    return lin_J, lin_nabla_J

q = q_0
d = q.copy()
for i in range(10000):
    lin_J, lin_nabla_J = compute(q, d, 1)
    d -= lin_nabla_J.to_numpy()
    print(lin_J)


39.72254200968959
39.449339728806606
39.17714098371909
38.905945775593786
38.63575410559743
38.36656597489672
38.09838138465842
37.83120033604924
37.565022830235925
37.299848868385226
37.03567845166384
36.77251158123854
36.51034825827606
36.24918848394314
35.98903225940651
35.72987958583293
35.471730464389154
35.2145848962419
34.958442882557954
34.70330442450403
34.44916952324691
34.19603817995333
33.943910395790056
33.692786171923835
33.44266550952144
33.19354840974962
32.945434873775135
32.698324902764746
32.45221849788523
32.207115660303344
31.963016391185846
31.719920691699514
31.47782856301114
31.236740006287448
30.99665502269525
30.757573613401316
30.51949577957241
30.28242152237532
30.046350842976814
29.811283742543676
29.5772202222427
29.34416028324065
29.11210392670433
28.88105115380052
28.651001965696004
28.421956363557577
28.193914348552024
27.966875921846146
27.740841084606743
27.51580983800059
27.29178218319449
27.06875812135526
26.84673765364968
26.625720781244553
26.4057


KeyboardInterrupt



In [19]:
J, nabla_J = compute(q_0)
print(J)
print(nabla_J)

0.0012500000000000011
[[1.06904994e-09 7.52967833e-09 1.43982664e-08 ... 1.46831147e-08
  7.21949784e-09 1.03216528e-09]
 [1.19224861e-09 8.61697352e-09 1.68575706e-08 ... 1.70338053e-08
  8.18605960e-09 1.14028573e-09]
 [1.17578103e-09 8.64405623e-09 1.70945864e-08 ... 1.72289045e-08
  8.18691109e-09 1.12010707e-09]
 ...
 [8.01172210e-10 6.70063988e-09 1.39910581e-08 ... 1.42122430e-08
  6.36392196e-09 7.57160227e-10]
 [6.88481214e-10 6.03854113e-09 1.27737958e-08 ... 1.29738738e-08
  5.72016666e-09 6.46565190e-10]
 [5.25417269e-10 4.86748405e-09 1.02865415e-08 ... 1.03275491e-08
  4.52685347e-09 4.82370768e-10]]


In [11]:
np.max(nabla_J.to_numpy())

1.4728911112852965e-07

In [28]:
# q = q_0
# for i in range(10000):
#     J, nabla_J = compute(q)
#     q -= 1e5 * nabla_J.to_numpy()
#     print(J)



In [36]:
J, nabla_J = compute(q_0 - 1e1 * nabla_J.to_numpy())
print(J)
#print(nabla_J)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

0.00016074369508906668


In [59]:
J, nabla_J = compute(q_exact)
print(J)
#print(nabla_J)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

-8.131516293641283e-20
